# Hello Node Kubernetes

**Learning Objectives**
 * Create a Node.js server
 * Create a Docker container image
 * Create a container cluster and a Kubernetes pod
 * Scale up your services

## Overview

The goal of this hands-on lab is for you to turn code that you have developed into a replicated application running on Kubernetes, which is running on Kubernetes Engine. For this lab the code will be a simple Hello World node.js app.

Here's a diagram of the various parts in play in this lab, to help you understand how the pieces fit together with one another. Use this as a reference as you progress through the lab; it should all make sense by the time you get to the end (but feel free to ignore this for now).

<img src='../assets/k8s_hellonode_overview.png' width='50%'>

## Setup

To run this lab, you may need to run the following command:

```bash
../../../scripts/setup_kubernetes_auth.sh
```

## Create a Node.js server

The file `./src/server.js` contains a simple Node.js server. Use `cat` to examine the contents of that file.

In [ ]:
!cat ./src/server.js

Start the server by running `node server.js` in the cell below. Open a terminal and type
```bash
curl http://localhost:8000
```
to see what the server outputs. 

In [ ]:
!node ./src/server.js

You should see the output `"Hello World!"`. Once you've verfied this, interupt the above running cell by hitting the stop button.

## Create and build a Docker image

Now we will create a docker image called `hello_node.docker` that will do the following:

 1. Start from the node image found on the Docker hub by inhereting from `node:6.9.2`
 2. Expose port 8000
 3. Copy the `./src/server.js` file to the image
 4. Start the node server as we previously did manually

Save your Dockerfile in the folder labeled `dockerfiles`. Your finished Dockerfile should look something like this


```bash
FROM node:6.9.2

EXPOSE 8000

COPY ./src/server.js .

CMD node server.js
```

Next, build the image in your project using `docker build`.

In [ ]:
import os

PROJECT_ID = "qwiklabs-asl-02-3975462eb556"  # REPLACE WITH YOUR PROJECT NAME
ARTIFACT_REGISTRY_DIR = "asl-artifact-repo"
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["ARTIFACT_REGISTRY_DIR"] = ARTIFACT_REGISTRY_DIR

In [ ]:
%%bash
docker build -f dockerfiles/hello_node.docker -t us-docker.pkg.dev/${PROJECT_ID}/${ARTIFACT_REGISTRY_DIR}/hello-node:v1 .

It'll take some time to download and extract everything, but you can see the progress bars as the image builds. Once complete, test the image locally by running a Docker container as a daemon on port 8000 from your newly-created container image.

Run the container using `docker run`

In [ ]:
%%bash
docker run -d -p 8000:8000 us-docker.pkg.dev/${PROJECT_ID}/${ARTIFACT_REGISTRY_DIR}/hello-node:v1

Your output should look something like this:
```bash
b16e5ccb74dc39b0b43a5b20df1c22ff8b41f64a43aef15e12cc9ac3b3f47cfd
```

Right now, since you used the `--d` flag, the container process is running in the background. You can verify it's running using `curl` as before.

In [ ]:
!curl http://localhost:8000

Now, stop running the container. Get the container id using `docker ps` and then terminate using `docker stop`

In [ ]:
!docker ps

In [ ]:
# your container id will be different
!docker stop 2760d59ec659

Now that the image is working as intended, push it to the Google Artifact Registry, a private repository for your Docker images, accessible from your Google Cloud projects. First, configure docker uisng your local config file. The initial push may take a few minutes to complete. You'll see the progress bars as it builds.

In [ ]:
%%bash
gcloud auth print-access-token | docker login -u oauth2accesstoken --password-stdin https://us-docker.pkg.dev

In [ ]:
%%bash
docker push us-docker.pkg.dev/${PROJECT_ID}/${ARTIFACT_REGISTRY_DIR}/hello-node:v1

The container image will be listed in your Console. Select `Navigation` > `Artifact Registry`.

## Create a cluster on GKE

Now you're ready to create your Kubernetes Engine cluster. A cluster consists of a Kubernetes master API server hosted by Google and a set of worker nodes. The worker nodes are Compute Engine virtual machines.

Create a cluster with two n1-standard-1 nodes (this will take a few minutes to complete). You can safely ignore warnings that come up when the cluster builds.

**Note**: You can also create this cluster through the Console by opening the Navigation menu and selecting `Kubernetes Engine` > `Kubernetes clusters` > `Create cluster`.

In [ ]:
!gcloud container clusters create hello-world \
                --num-nodes 2 \
                --machine-type n1-standard-1 \
                --zone us-central1-a

## Create a pod

A Kubernetes pod is a group of containers tied together for administration and networking purposes. It can contain single or multiple containers. Here you'll use one container built with your `Node.js` image stored in your private Artifact Registry. It will serve content on port 8000.

Create a pod with the `kubectl run` command.

In [ ]:
%%bash
kubectl create deployment hello-node \
    --image=us-docker.pkg.dev/${PROJECT_ID}/${ARTIFACT_REGISTRY_DIR}/hello-node:v1

As you can see, you've created a deployment object. Deployments are the recommended way to create and scale pods. Here, a new deployment manages a single pod replica running the `hello-node:v1` image.

View the deployment using `kubectl get`

In [ ]:
!kubectl get deployments

Similarly, view the pods created by the deployment by also using `kubectl get`

In [ ]:
!kubectl get pods

Here are some other good kubectl commands you should know. They won't change the state of the cluster. Others can be found [here](https://kubernetes.io/docs/reference/kubectl/overview/).
 * `kubectl cluster-info`
 * `kubectl config view`
 * `kubectl get events`
 * `kubectl logs <pod-name>`

## Allow external traffic

By default, the pod is only accessible by its internal IP within the cluster. In order to make the hello-node container accessible from outside the Kubernetes virtual network, you have to expose the pod as a Kubernetes service.

You can expose the pod to the public internet with the `kubectl expose` command. The `--type="LoadBalancer"` flag is required for the creation of an externally accessible IP. This flag specifies that we are using the load-balancer provided by the underlying infrastructure (in this case the Compute Engine load balancer). Note that you expose the deployment, and not the pod, directly. This will cause the resulting service to load balance traffic across all pods managed by the deployment (in this case only 1 pod, but you will add more replicas later).

In [ ]:
!kubectl expose deployment hello-node --type="LoadBalancer" --port=8000

The Kubernetes master creates the load balancer and related Compute Engine forwarding rules, target pools, and firewall rules to make the service fully accessible from outside of Google Cloud.

To find the publicly-accessible IP address of the service, request kubectl to list all the cluster services.

In [ ]:
!kubectl get services

There are 2 IP addresses listed for your `hello-node` service, both serving port 8000. The `CLUSTER-IP` is the internal IP that is only visible inside your cloud virtual network; the `EXTERNAL-IP` is the external load-balanced IP.

You should now be able to reach the service by calling `curl http://<EXTERNAL_IP>:8000`.

In [ ]:
!curl http://35.223.2.76:8000

At this point you've gained several features from moving to containers and Kubernetes - you do not need to specify on which host to run your workload and you also benefit from service monitoring and restart. Now see what else can be gained from your new Kubernetes infrastructure.

## Scale up your service
One of the powerful features offered by Kubernetes is how easy it is to scale your application. Suppose you suddenly need more capacity. You can tell the replication controller to manage a new number of replicas for your pod: 
```bash
kubectl scale deployment hello-node --replicas=4
```

Scale your `hello-node` application to have 6 replicas. Then use `kubectl get` to request a description of the updated deployment and list all the pods:

In [ ]:
!kubectl scale deployment hello-node --replicas=6

In [ ]:
!kubectl get deployment

In [ ]:
!kubectl get pods

A declarative approach is being used here. Rather than starting or stopping new instances, you declare how many instances should be running at all times. Kubernetes reconciliation loops makes sure that reality matches what you requested and takes action if needed.

Here's a diagram summarizing the state of your Kubernetes cluster:

<img src='../assets/k8s_cluster.png' width='60%'>

## Roll out an upgrade to your service
At some point the application that you've deployed to production will require bug fixes or additional features. Kubernetes helps you deploy a new version to production without impacting your users.

First, modify the application by opening `server.js` so that the response is 
```bash
response.end("Hello Kubernetes World!");
```

Now you can build and publish a new container image to the registry with an incremented tag (`v2` in this case).

**Note**: Building and pushing this updated image should be quicker since caching is being taken advantage of.

In [ ]:
%%bash
docker build -f dockerfiles/hello_node.docker -t us-docker.pkg.dev/${PROJECT_ID}/${ARTIFACT_REGISTRY_DIR}/hello-node:v2 .
docker push us-docker.pkg.dev/${PROJECT_ID}/${ARTIFACT_REGISTRY_DIR}/hello-node:v2

Kubernetes will smoothly update your replication controller to the new version of the application. In order to change the image label for your running container, you will edit the existing `hello-node` deployment and change the image from `us-docker.pkg.dev/[PROJECT_ID]/[ARTIFACT_REGISTRY_DIR]/hello-node:v1` to `us-docker.pkg.dev/[PROJECT_ID]/[ARTIFACT_REGISTRY_DIR]/hello-node:v2`.

To do this, use the `kubectl edit` command. It opens a text editor displaying the full deployment yaml configuration. It isn't necessary to understand the full yaml config right now, just understand that by updating the `spec.template.spec.containers.image` field in the config you are telling the deployment to update the pods with the new image.

Open a terminal and run the following command:

```bash 
kubectl edit deployment hello-node
```

Look for `Spec` > `containers` > `image` and change the version number to `v2`. This is the output you should see:

```bash
deployment.extensions/hello-node edited
```

New pods will be created with the new image and the old pods will be deleted. Run `kubectl get deployments` to confirm. 

**Note**: You may need to rerun the above command as it provisions machines.

In [ ]:
!kubectl get deployments

While this is happening, the users of your services shouldn't see any interruption. After a little while they'll start accessing the new version of your application. You can find more details on rolling updates in this documentation.

Hopefully with these deployment, scaling, and updated features, once you've set up your Kubernetes Engine cluster, you'll agree that Kubernetes will help you focus on the application rather than the infrastructure.

## Cleanup

Delete the cluster using `gcloud` to free up those resources. Use the `--quiet` flag if you are executing this in a notebook. Deleting the cluster can take a few minutes. 

In [ ]:
!gcloud container clusters --quiet delete hello-world

Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.